# Visualize examples of Memento10k

In [1]:
# Imports
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
import cv2
from cv2 import imread, resize
from scipy.stats import spearmanr
from keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, GlobalAveragePooling2D, Input, TimeDistributed
from keras.models import Model
from keras.utils import Sequence
import keras.backend as K
from keras.optimizers import Adam, SGD
from timeit import default_timer as timer
import keras
import tensorflow as tf
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam
import pickle
import json
from tqdm import tqdm_notebook as tqdm
import seaborn as sns

sys.path.append('../src/')
sys.path.append('../imgaug')
from generator import VideoSeqGenerator, preprocess_i3d_rgb
from vid_utils import load_video_opencv, plot_frames
from imgaug import augmenters as iaa
from imgaug import parameters as iap

import i3d_config as cfg
from keras_models import build_i3d_custom, build_model_multigpu

%load_ext autoreload
%autoreload 2


Using TensorFlow backend.


In [3]:
print(os.uname()[1])

cfosco-lambda


In [ ]:
# Set data paths

data_path = '../../moments/10k_clean'
labels_path = '../../moments/split'
names_with_slash = True
load_func = 'vids_opencv'

# TRAIN_ON = 'rgb'

if os.uname()[1] == 'visiongpu30':
    data_path = '../../../moments_raw/sources_numpy_256/' if TRAIN_ON=='rgb' else '../../../moments_flow/sources_flow_down1_tvl1_skip1/'
    labels_path = '../memento10k_labels/'
    names_with_slash = False
    load_func = 'npy_'+TRAIN_ON
else:
    data_path = '../../memento'
    labels_path = '../../memento_data'
    names_with_slash = True
    load_func = 'vids_opencv'


with open(os.path.join(labels_path, 'train_test_split_moments.pkl'), 'rb') as f:
    tt_s = pickle.load(f)
    
train_names=[]
val_names=[]
test_names=[]

# Fetch scores from ground truth json file with a defined T
T = 180
mem_scores_file = 'normalized_mem_scores_ma_T=%d.json' % T
alpha_file = 'alphas_ma_T=%d.json' % T

with open(os.path.join(labels_path, mem_scores_file)) as f:
    name_to_mem = json.load(f)
    
with open(os.path.join(labels_path, alpha_file)) as f:
    name_to_alpha = json.load(f)
    
# Getting dict in same format as vid names




name_to_mem_alpha={}

if names_with_slash:
    for n in tt_s[0]:
        cl = n.split('_')[0]
        cl = cl.replace('-','+')
        newn = cl+'/'+'_'.join(n.split('_')[1:])
        train_names.append(newn)

    for n in tt_s[1]:
        cl = n.split('_')[0]
        cl = cl.replace('-','+')
        newn = cl+'/'+'_'.join(n.split('_')[1:])
        val_names.append(newn)
        
    for k,v in name_to_mem.items():
        name_to_mem_alpha[k] = np.array([v, name_to_alpha[k]])
    
else:
    train_names = sorted([ n[:-4]+'.npy' for n in tt_s[0] ])
    val_names = sorted([ n[:-4]+'.npy' for n in tt_s[1] ])
    for k,v in name_to_mem.items():
        newk = k.replace('/','_').replace('+','-')[:-4]+'.npy' 
        name_to_mem_alpha[newk] = np.array([v, name_to_alpha[k]])
    
all_names_set = set(list(name_to_mem_alpha.keys()))
test_names = list(all_names_set - set(train_names) - set(val_names))
    


print(train_names[:5])
print(list(name_to_mem_alpha.keys())[:5])


# Check if all names are in dict
to_remove=[]
for tn in train_names:
    if not os.path.isfile(os.path.join(data_path, tn)):
        to_remove.append(tn)
    if tn not in name_to_mem_alpha.keys():
        print('Train vid not in dict:',tn)
        
if to_remove:
    for r in to_remove:
        train_names.remove(to_remove)
        
to_remove=[]
for tn in val_names:
    if not os.path.isfile(os.path.join(data_path, tn)):
        to_remove.append(tn)
    if tn not in name_to_mem_alpha.keys():
        print('Val vid not in dict:',tn)
    
if to_remove:
    for r in to_remove:
        val_names.remove(to_remove)

to_remove=[]
for tn in test_names:
    if not os.path.isfile(os.path.join(data_path, tn)):
        to_remove.append(tn)
    if tn not in name_to_mem_alpha.keys():
        print('Test vid not in dict:',tn)

if to_remove:
    for r in to_remove:
        test_names.remove(r)

print('Loaded %d train videos, %d val videos, %d test videos\n' % (len(train_names), len(val_names), len(test_names)))


In [ ]:
# Instantiate generator

gen_train_rgb = VideoSeqGenerator(files=train_names,
                            batch_size=bs, 
                            has_label_col=False, 
                            dataset_path=data_path,
                            label_csv= None, #'../../moments/split/moments_mini_categories.txt',
                            augment=None,
                            load_func=load_func,
                            load_labels_func='mem_alpha',
                            preprocess_func='fast_'+TRAIN_ON,
                            remove_excess_files=False,
                            shuffle=False,
                            is_train=False,
                            it_per_epoch=None,
                            return_labels = True,
                            verbose=True,
                            str2label_dict = name_to_mem_alpha)

gen_train_flow = VideoSeqGenerator(files=train_names,
                            batch_size=bs, 
                            has_label_col=False, 
                            dataset_path=data_path,
                            label_csv= None, #'../../moments/split/moments_mini_categories.txt',
                            augment=None,
                            load_func=load_func,
                            load_labels_func='mem_alpha',
                            preprocess_func='fast_'+TRAIN_ON,
                            remove_excess_files=False,
                            shuffle=False,
                            is_train=False,
                            it_per_epoch=None,
                            return_labels = True,
                            verbose=True,
                            str2label_dict = name_to_mem_alpha)

In [2]:
# Test generator

k = 0

batch, labels = gen_train_rgb.__getitem__(k)
batch, labels = gen_train_flow.__getitem__(k)

# Display
for i,b in enumerate(batch):
    mem, alpha = labels[i]
    plot_frames(b, 
                title='%s | %d frames. STM: %s. Alpha: %s' % ('vid',len(b),mem,alpha),
                is_optical_flow=True if TRAIN_ON=='flow' else False,
                suptitle_y=0.89,
                is_255image=True, 
                frames_to_show=4)